<a href="https://colab.research.google.com/github/AkindeKadjo/MNIST_digits_Image_Classification/blob/main/02_PCA_Excercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Akinde Kadjo

Date: 10/17/2002

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

#1.Load the Data

In [ ]:
# load the dataset
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
# view the shape of the dataset
mnist.data.shape

(70000, 784)

In [ ]:
# set `mnist.data` as X
X = mnist.data.copy()
X.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# set `mnist.target` as y
y = mnist.target.copy()
y.head()

0    5
1    0
2    4
3    1
4    9
Name: class, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [ ]:
#combine both as a single df
df= pd.concat([X, y],axis = 1)
df.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9


In [ ]:
# how many data points are duplicated?
df.duplicated().sum()

0

In [ ]:
#check for missing values
df.isna().sum().sum()

0

In [ ]:
# Explore the class balance in the target column.
df['class'].value_counts(normalize=True)

1    0.112529
7    0.104186
3    0.102014
2    0.099857
9    0.099400
0    0.098614
6    0.098229
8    0.097500
4    0.097486
5    0.090186
Name: class, dtype: float64

#2. Prepare the Data

Prepare the data for modeling. Scale and apply PCA to your data, while retaining 95% of the variance. Be sure not to leak information.

In [ ]:
#Performing a train test split, here the random state of 42 is chosen for concistency
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 42)

In [ ]:
# Instantiate Standard Scaler
scaler = StandardScaler()
# Create a PCA object that will retain 95% of the variance when transforming
pca = PCA(n_components=.95)
# Combine the scaler and the PCA in a pipeline.
pca_pipe = make_pipeline(scaler, pca)
# Transform the training data and check shape of new features after applying PCA
pca_pipe.fit_transform(X_train).shape[1]

328

In [ ]:
X_train.shape[1]

784

#3. Create 2 KNN models. & 4. Evaluate and compare the models.

a. One that that uses the PCA transformed data to predict which number each image shows.

In [ ]:
%%time
# Create and fit a KNN model WITH PCA.
knn_pca = make_pipeline(scaler, pca, KNeighborsClassifier())
knn_pca.fit(X_train, y_train)

CPU times: user 11.8 s, sys: 866 ms, total: 12.7 s
Wall time: 7.47 s


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('pca', PCA(n_components=0.95)),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [ ]:
%%time
# Make predictions and evaluate the model using classification_report
pca_preds = knn_pca.predict(X_test)
print(classification_report(y_test, pca_preds))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1714
           1       0.96      0.99      0.97      1977
           2       0.95      0.94      0.94      1761
           3       0.94      0.94      0.94      1806
           4       0.94      0.94      0.94      1587
           5       0.95      0.93      0.94      1607
           6       0.96      0.98      0.97      1761
           7       0.94      0.93      0.94      1878
           8       0.97      0.90      0.93      1657
           9       0.91      0.93      0.92      1752

    accuracy                           0.95     17500
   macro avg       0.95      0.95      0.95     17500
weighted avg       0.95      0.95      0.95     17500

CPU times: user 42.5 s, sys: 1.4 s, total: 43.9 s
Wall time: 29.5 s


b. One that uses the original data, without the PCA transformation (but, remember you still need to scale the data!)

In [ ]:
%%time
# Create and fit a KNN model WITHOUT PCA.
knn_pipe = make_pipeline(scaler, KNeighborsClassifier())
knn_pipe.fit(X_train, y_train)

CPU times: user 378 ms, sys: 5.03 ms, total: 383 ms
Wall time: 383 ms


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kneighborsclassifier', KNeighborsClassifier())])

In [ ]:
%%time
# Make predictions and evaluate the model using classification_report
no_pca_preds = knn_pipe.predict(X_test)
print(classification_report(y_test, no_pca_preds))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1714
           1       0.95      0.99      0.97      1977
           2       0.95      0.93      0.94      1761
           3       0.93      0.94      0.94      1806
           4       0.94      0.93      0.94      1587
           5       0.94      0.93      0.94      1607
           6       0.96      0.97      0.97      1761
           7       0.94      0.93      0.93      1878
           8       0.97      0.89      0.93      1657
           9       0.90      0.92      0.91      1752

    accuracy                           0.94     17500
   macro avg       0.94      0.94      0.94     17500
weighted avg       0.94      0.94      0.94     17500

CPU times: user 1min 19s, sys: 2.09 s, total: 1min 21s
Wall time: 48.9 s


#5. Answer the Following Questions in Text:

a. Which model performed the best on the test set?


>PCA performed better with an accuracy of 0.95 versus without that just had an accuracy of 0.94

b. Which model was the fastest at making predictions?

>Total (fitting + report) time without PCA: 81.4 seconds

>Total (fitting + report) time with PCA: 62.6 seconds

PCA was the fastest at making predictions